In [1]:
from model_utils import ConvNext_wrapper
import os
from torchvision import transforms
from src.data.our_kaggle_food_dataset import KaggleFoodDataset
from torch.utils.data import DataLoader
import torch
import tqdm

d = "cuda"
with torch.no_grad():
    model = ConvNext_wrapper(classification=True)
    model = model.to(d)
    model.d = d
    
    current_dir = r"C:\Users\Karlu\Desktop\DeliNet"
    food_dir = os.path.join(current_dir,'data','processed','KaggleFoodDataset')
    csv_file_path = os.path.join(food_dir, 'data.csv')
    image_dir = os.path.join(food_dir,'images')
    batch_size = 200
    preprocess = transforms.Compose([
        transforms.Resize((224,224)),
        #transforms.CenterCrop(224, 224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
    
    data_set_with_transf = KaggleFoodDataset(csv_file=csv_file_path, image_dir=image_dir,
                                             transform=preprocess, train=True, train_split=1.0)
    
    dataloader_test = DataLoader(data_set_with_transf, batch_size=batch_size, shuffle=False)
    
    preds = []
    for img, _ in tqdm.tqdm(dataloader_test):
        img = img.to(d)
        pred = model(img)
        preds.append(pred)


100%|██████████| 68/68 [01:44<00:00,  1.53s/it]


In [3]:
preds_ = torch.concat(preds, dim=0)
preds_ = preds_.argmax(dim=1)

In [5]:
preds_ = preds_.cpu().numpy()

In [13]:
import numpy as np
preds_uniq, preds_count = np.unique(preds_, return_counts=True)


In [26]:
largest_classes = preds_uniq[np.flip(preds_count.argsort())]

all_classes_string = ""
for largest_class in largest_classes:
    all_classes_string += " -- " + model.id2label[largest_class]

all_classes_string

" -- bubble_and_squeak -- piece_de_resistance -- chicken_casserole -- fried_rice, Chinese_fried_rice -- tablefork -- salad_bar -- bacon_and_eggs -- refreshment -- club_sandwich, three-decker, triple-decker -- antipasto -- coffee_break, tea_break -- charlotte -- diet -- silage, ensilage -- soft_diet, pap, spoon_food -- tea, afternoon_tea, teatime -- salad -- shrimp -- frog_legs -- diet -- apricot_sauce -- zabaglione, sabayon -- souvlaki, souvlakia -- schnitzel, Wiener_schnitzel -- souffle -- molded_salad -- turkey_stuffing -- stuffing, dressing -- pepper_steak -- snail_butter -- relish -- wrap -- snowball -- chocolate_pudding -- hamburger_bun, hamburger_roll -- canary_seed -- adobo -- open-face_sandwich, open_sandwich -- loblolly -- puff_batter, pouf_paste, pate_a_choux -- tossed_salad -- mushroom -- chicory, curly_endive -- toast -- combination_salad -- chicken_salad -- pina_colada -- macadamia_nut -- chicken_Kiev -- Jerusalem_artichoke, sunchoke -- pasta -- perishable, spoilable -- fr